In [1]:
import numpy as np
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
[WinError 182] The operating system cannot run %1. Error loading "C:\Users\me1awq\.conda\envs\cpu\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
# Load the pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('all-mpnet-base-v2', num_labels=1) # Change num_labels to 1 for regression
tokenizer = BertTokenizer.from_pretrained('all-mpnet-base-v2 ')



# Load the CSV file
data = pd.read_csv(r"C:/Users/me1awq/PhD/docsim/datasets/rssb/mpnet_result.csv")

In [ ]:
# Split the data into train, validation, and test sets
train_data, val_data, test_data = np.split(data.sample(frac=1, random_state=42), [int(.6*len(data)), int(.8*len(data))])

# Tokenize the texts
train_encodings = tokenizer(train_data['Target Rule'].tolist(), train_data['Source Rule'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_data['Target Rule'].tolist(), val_data['Source Rule'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_data['Target Rule'].tolist(), test_data['Source Rule'].tolist(), truncation=True, padding=True)

# Convert the tokenized data into PyTorch tensors
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), torch.tensor(train_data['Cosine Similarity'].tolist()))
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']), torch.tensor(val_encodings['attention_mask']), torch.tensor(val_data['Cosine Similarity'].tolist()))
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']), torch.tensor(test_encodings['attention_mask']), torch.tensor(test_data['Cosine Similarity'].tolist()))

# Define the data loader for training, validation, and test
batch_size = 15
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)
loss_fn = torch.nn.MSELoss()

In [ ]:
# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)
loss_fn = torch.nn.MSELoss()

# Train the model
epochs = 30
train_losses = []
val_losses = []
test_losses = []
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    for batch in val_dataloader:
        input_ids, attention_mask, labels = batch
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels.view(-1,1))
        val_loss += loss.item()
        preds = outputs.logits.view(-1).cpu().numpy()
        val_preds.extend(preds)
        val_labels.extend(labels.cpu().numpy())
    val_mse = mean_squared_error(val_labels, val_preds)

    # Store the losses for plotting
    train_losses.append(train_loss/len(train_dataloader))
    val_losses.append(val_loss/len(val_dataloader))

    print("Epoch {} Train loss: {:.4f} Validation loss: {:.4f} Validation MSE: {:.4f}".format(epoch+1, train_loss/len(train_dataloader), val_loss/len(val_dataloader), val_mse))

    # Test the model
    model.eval()
    test_loss = 0
    test_preds = []
    test_labels = []
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels.view(-1,1))
        test_loss += loss.item()
        preds = outputs.logits.view(-1).cpu().numpy()
        test_preds.extend(preds)
        test_labels.extend(labels.cpu().numpy())

    # Store the losses for plotting
    test_losses.append(test_loss/len(test_dataloader))

    test_mse = mean_squared_error(test_labels, test_preds)
    print("Epoch {} Train loss: {:.4f} Test loss: {:.4f} Test MSE: {:.4f}".format(epoch+1, train_loss/len(train_dataloader), test_loss/len(test_dataloader), test_mse))



In [ ]:
# Plot the train, validation, and test losses
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Plot the train, validation, and test losses
plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()